In [ ]:
import pandas as pd
import altair as alt
from sublimpy import utils
alt.data_transformers.enable('json')

ls ~/Development/data/sublimationofsnow/ | grep csv

In [ ]:
src = pd.read_csv(
    "~/Development/data/sublimationofsnow/monthly_planar_fits.csv", delim_whitespace=True
).groupby(['height', 'tower']).mean().sort_values('tilt').reset_index()

src[src.height.isin(['3','10'])]

In [ ]:
df_nopf = pd.read_parquet("tidy_df_20221101_20230619_noplanar_fit.parquet")
df_pf = pd.read_parquet("tidy_df_20221101_20230619_planar_fit.parquet")
df_pfmp = pd.read_parquet("tidy_df_20221101_20230619_planar_fit_multiplane.parquet")

df_nopf = utils.modify_df_timezone(df_nopf, 'UTC', 'US/Mountain')
df_pf = utils.modify_df_timezone(df_pf, 'UTC', 'US/Mountain')
df_pfmp = utils.modify_df_timezone(df_pfmp, 'UTC', 'US/Mountain')

df = pd.concat([
    df_nopf.query("measurement == 'w'").assign(type = "nopf"),
    df_pf.query("measurement == 'w'").assign(type = "pf"),
    df_pfmp.query("measurement == 'w'").assign(type = "pfmp"),
])

In [ ]:
df.query("type == 'pfmp'").query("tower == 'c'")

In [ ]:
import datetime as dt
import numpy as np

In [61]:
src_pfmp = df.query("type == 'pfmp'").set_index('time')
src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
src_pfmp.index = src_pfmp.index.set_names(['time','height','tower', 'type'])
src_pfmp = src_pfmp.reset_index()
src_pfmp['time'] = src_pfmp['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))

src_nopf = df.query("type == 'nopf'").set_index('time')
src_nopf = src_nopf.groupby([src_nopf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
src_nopf.index = src_nopf.index.set_names(['time','height','tower', 'type'])
src_nopf = src_nopf.reset_index()
src_nopf['time'] = src_nopf['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))

src_pf = df.query("type == 'pf'").set_index('time')
src_pf = src_pf.groupby([src_pf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
src_pf.index = src_pf.index.set_names(['time','height','tower', 'type'])
src_pf = src_pf.reset_index()
src_pf
src_pf['time'] = src_pf['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/1236953920.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/1236953920.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_nopf = src_nopf.groupby([src_nopf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/1236953920.py:14: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pf = src_pf.groupby([src_pf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()


In [86]:
line = alt.Chart().transform_calculate(y = '0').mark_rule().encode(y='y:Q')
plot = alt.Chart().mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('value:Q'),
    alt.Color('height:O').scale(scheme='turbo'),
)

In [87]:
alt.layer(
    line, plot, data = src_pfmp
).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates').display(renderer='svg')


alt.FacetChart(...)

In [66]:
(
    alt.layer(
        line, plot, data = src_pfmp
    ).properties(width = 150, height = 150).facet('tower:N', title='Streamwise coordinates')
    &
    alt.layer(
        line, plot, data = src_pf
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates')

    &
    alt.layer(
        line, plot, data = src_nopf
    ).properties(width = 150, height = 150).facet('tower:N', title='Geographic coordinates')

).display(renderer='svg')

alt.VConcatChart(...)

In [74]:

# Identify lists of timestamps for different categories
bs_times = set(
    df_pf.query("variable == 'SF_avg_1m_ue'").query("value > 0").time
).union(
    set(df_pf.query("variable == 'SF_avg_2m_ue'").query("value > 0").time)
)
nobs_times = set(df_pf.time).difference(bs_times)

ri_stable_times = df_pf.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = df_pf.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = df_pf.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = df_pf.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = df_pf.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = df_pf.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

In [98]:
def w_diurnal_chart(src, title):
    return alt.layer(
        line, plot.encode(alt.Y('value:Q').scale(domain=[-0.05,0.05])), data = src
    ).properties(width = 150, height = 150).properties(title=title)

def w_diurnal_df(filter_times):
    src_pfmp = df.query("type == 'pfmp'").set_index('time').query("tower == 'c'")
    src_pfmp = src_pfmp[src_pfmp.index.isin(filter_times)]
    src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
    src_pfmp.index = src_pfmp.index.set_names(['time','height','tower', 'type'])
    src_pfmp = src_pfmp.reset_index()
    src_pfmp['time'] = src_pfmp['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))
    return src_pfmp

(
w_diurnal_chart(w_diurnal_df(df_pf.time), 'All times') |\
w_diurnal_chart(w_diurnal_df(ri_stable_times), 'Stable times') |\
w_diurnal_chart(w_diurnal_df(ri_unstable_times), 'Unstable times') |\
w_diurnal_chart(w_diurnal_df(ri_neutral_times), 'Neutral times')
).resolve_scale(y='shared', x='shared').configure_title(anchor='middle').interactive().display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/991269909.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/991269909.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/991269909.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_47932/991269909.py:9: FutureWarning: 'T' is deprecated and 

alt.HConcatChart(...)

In [ ]:
start_date = '20221101'
end_date = '20230619'

def daily_w_cycle(src, title, ylim):
    return alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(value):Q').title('Wind speed (m/s)').scale(domain=ylim),
        alt.Color('height:O').scale(scheme='sinebow'),
        alt.StrokeDash('tower:N'),
        tooltip='variable',
    ).properties(width = 200, height = 200, title=title)

(

    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet"),
        'SRLEC',
        ylim=[-0.10, 0.05]
    ) | daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"),
        'Geographic',
        ylim=[-0.10, 0.05]
    ) | 
    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet"),
        'Streamwise',
        ylim=[-0.10, 0.05]
    )
).properties(title='Vertical wind speed, different coordinate systems').display(renderer='svg')